In [31]:
from dotenv import load_dotenv
import os
import json
import textwrap

from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.docstore.document import Document
from langchain.chains import GraphCypherQAChain
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms import Ollama
from langchain_community.graphs.index_creator import GraphIndexCreator

from langchain_openai import OpenAI


# Warning control
import warnings
warnings.filterwarnings("ignore")

In [ ]:
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_ENDPOINT = 'https://api.openai.com/v1/embeddings'

VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

In [40]:
graph = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [45]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4o-mini", temperature=0)
# Instantiate the LLM graph transformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [46]:
llm = Ollama(
    model="llama3",  
    temperature=0,
)
llm_transformer = LLMGraphTransformer(llm=llm)

In [47]:
docs = [
    Document(page_content='''
The 20th century witnessed the rise of some of the most influential scientists in history, with Albert Einstein and Marie Curie standing out among them. Einstein, best known for his theory of relativity, revolutionized our understanding of space, time, and energy, earning him the Nobel Prize in Physics in 1921 for his explanation of the photoelectric effect. Marie Curie, a pioneer in the study of radioactivity, was the first woman to win a Nobel Prize. She was awarded the Nobel Prize in'''),
    Document(page_content=''' radioactivity, was the first woman to win a Nobel Prize. She was awarded the Nobel Prize in Physics in 1903, shared with her husband Pierre Curie and Henri Becquerel, for their work on radiation. Curie later made history again by winning a second Nobel Prize in Chemistry in 1911 for her discoveries of radium and polonium. Both scientists made monumental contributions that continue to influence the fields of physics and beyond.
''')
]
docs = [
    Document(page_content="The capital of France is Paris."),
    Document(page_content="Python is a popular programming language."),
]

In [48]:
graph_documents = llm_transformer.convert_to_graph_documents(docs)
print(f"Derived Nodes:\n{graph_documents[0].nodes}\n")
print(f"Derived Edges:\n{graph_documents[0].relationships}")

Derived Nodes:
[Node(id='Paris', type='City', properties={}), Node(id='France', type='Country', properties={})]

Derived Edges:
[Relationship(source=Node(id='France', type='Country', properties={}), target=Node(id='Paris', type='City', properties={}), type='HAS_CAPITAL', properties={})]


In [49]:
print(graph_documents)

[GraphDocument(nodes=[Node(id='Paris', type='City', properties={}), Node(id='France', type='Country', properties={})], relationships=[Relationship(source=Node(id='France', type='Country', properties={}), target=Node(id='Paris', type='City', properties={}), type='HAS_CAPITAL', properties={})], source=Document(metadata={}, page_content='The capital of France is Paris.')), GraphDocument(nodes=[Node(id='Python', type='Language', properties={}), Node(id='programming', type='Activity', properties={})], relationships=[Relationship(source=Node(id='Python', type='Language', properties={}), target=Node(id='programming', type='Activity', properties={}), type='USED_FOR', properties={})], source=Document(metadata={}, page_content='Python is a popular programming language.'))]


In [2]:
index_creator = GraphIndexCreator(llm=llm)
text = 'It won''t look like much, but if you stop and look closely, you''ll see a “Field of dreams,” the ground on which America''s future will be built. \nThis is where Intel, the American company that helped build Silicon Valley, is going to build its $20 billion semiconductor “mega site”. \nUp to eight state-of-the-art factories in one place. 10,000 new good-paying jobs. '
graph = index_creator.from_text(text)


NameError: name 'GraphIndexCreator' is not defined

In [36]:
print(graph.get_triples())

[]


In [1]:
# Add the graph documents, sources, and include entity labels
graph.add_graph_documents(graph_documents, 
include_source=True,
baseEntityLabel=True
)
# Refresh the graph schema
graph.refresh_schema()

# Print the graph schema
print(graph.get_schema)

NameError: name 'graph' is not defined

In [ ]:
# Query the graph
results = graph.query("""
MATCH (relativity:Concept {id: "Theory Of Relativity"}) <-[:KNOWN_FOR]- (scientist)
RETURN scientist
""")

print(results[0])


In [ ]:
# Create the Graph Cypher QA chain
graph_qa_chain = GraphCypherQAChain.from_llm(
    llm=ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0), graph=graph, verbose=True
)
# Invoke the chain with the input provided
result = graph_qa_chain.invoke({"query": "Who discovered the element Radium?"})
print(result)
# Print the result text
print(f"Final answer: {result['result']}")